# RL Exercise 6 - Training with Ray and Serving with Clipper

**GOAL:** The goal of this exercise is to show how to train a policy with Ray and to deploy it with Clipper in a fun, interactive way.

We will train an agent to play Pong, and then we will play Pong against the policy that we trained.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gym
import pong_py
import ray

from ray.tune.registry import register_env
from ray.rllib.agents import ppo

Instructions for updating:
Use the retry module or similar alternatives.


Start Ray.

In [2]:
ray.init()

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:52284 to respond...
Waiting for redis server at 127.0.0.1:41463 to respond...
Starting local scheduler with the following resources: {'CPU': 16, 'GPU': 1}.

View the web UI at http://localhost:8889/notebooks/ray_ui62916.ipynb?token=6142f365f2bf1eaef4bba03888053967b12167d2d9b2b039



{'node_ip_address': '172.17.0.2',
 'redis_address': '172.17.0.2:52284',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store71471754', manager_name='/tmp/plasma_manager62828724', manager_port=53327)],
 'local_scheduler_socket_names': ['/tmp/scheduler21957333'],
 'raylet_socket_names': [],
 'webui_url': 'http://localhost:8889/notebooks/ray_ui62916.ipynb?token=6142f365f2bf1eaef4bba03888053967b12167d2d9b2b039'}

Instantiate an agent that can be trained using Proximal Policy Optimization (PPO).

In [4]:
def env_creator(env_config):
    return pong_py.PongJSEnv()

register_env("my_env", env_creator)
trainer = ppo.PPOAgent(env="my_env", config={
    "env_config": {},  # config to pass to env creator
})

Created LogSyncer for /home/jovyan/ray_results/2018-09-21_20-22-229s806msu -> None
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
LocalMultiGPUOptimizer devices ['/cpu:0']


Train the `PPOAgent` for some number of iterations.

**EXERCISE:** You will need to experiment with the number of iterations as well as with the configuration to get the agent to learn something reasonable.

In [5]:
for i in range(2):
    result = trainer.train()

== sgd epochs ==
0 {'cur_lr': 4.9999998736893758e-05, 'total_loss': 2298.363, 'policy_loss': -0.0010465819, 'vf_loss': 2298.364, 'vf_explained_var': 0.0017972627, 'kl': 4.816657e-05, 'entropy': 1.0985396}
1 {'cur_lr': 4.9999998736893758e-05, 'total_loss': 2296.1841, 'policy_loss': -0.0020927656, 'vf_loss': 2296.186, 'vf_explained_var': 0.0056349211, 'kl': 0.00021130528, 'entropy': 1.0983686}
2 {'cur_lr': 4.9999998736893758e-05, 'total_loss': 2294.9033, 'policy_loss': -0.0028287813, 'vf_loss': 2294.9062, 'vf_explained_var': 0.0091754002, 'kl': 0.00073410041, 'entropy': 1.0978278}
3 {'cur_lr': 4.9999998736893758e-05, 'total_loss': 2294.3191, 'policy_loss': -0.0038047591, 'vf_loss': 2294.3228, 'vf_explained_var': 0.0089636752, 'kl': 0.0014569559, 'entropy': 1.0970924}
4 {'cur_lr': 4.9999998736893758e-05, 'total_loss': 2293.7612, 'policy_loss': -0.004650441, 'vf_loss': 2293.7656, 'vf_explained_var': 0.0090345703, 'kl': 0.0026331909, 'entropy': 1.0959095}
5 {'cur_lr': 4.9999998736893758e-05

15 {'cur_lr': 4.9999998736893758e-05, 'total_loss': 2093.916, 'policy_loss': -0.0096406806, 'vf_loss': 2093.9238, 'vf_explained_var': 0.00436287, 'kl': 0.0092345923, 'entropy': 1.0863502}
16 {'cur_lr': 4.9999998736893758e-05, 'total_loss': 2093.915, 'policy_loss': -0.0099215694, 'vf_loss': 2093.9231, 'vf_explained_var': 0.0041438546, 'kl': 0.0096677803, 'entropy': 1.0863003}
17 {'cur_lr': 4.9999998736893758e-05, 'total_loss': 2093.915, 'policy_loss': -0.0099207554, 'vf_loss': 2093.9231, 'vf_explained_var': 0.0041349549, 'kl': 0.0087499479, 'entropy': 1.08655}
18 {'cur_lr': 4.9999998736893758e-05, 'total_loss': 2093.915, 'policy_loss': -0.0098973876, 'vf_loss': 2093.9229, 'vf_explained_var': 0.0041603167, 'kl': 0.0096352277, 'entropy': 1.0856956}
19 {'cur_lr': 4.9999998736893758e-05, 'total_loss': 2093.9146, 'policy_loss': -0.010132942, 'vf_loss': 2093.9229, 'vf_explained_var': 0.0040607341, 'kl': 0.0093232822, 'entropy': 1.0865316}
20 {'cur_lr': 4.9999998736893758e-05, 'total_loss': 20

Use the agent manually by calling `agent.compute_action` and see the rewards you get are consistent with the rewards printed during the training procedure.

In [6]:
env = pong_py.PongJSEnv()

for _ in range(20):
    state = env.reset()
    done = False
    cumulative_reward = 0

    while not done:
        action = trainer.compute_action(state)
        state, reward, done, _ = env.step(action)
        cumulative_reward += reward

    print(cumulative_reward)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
40
40
163
40
40
40
40
40
40
40
40
40
40
40
40
40
264
40
40
40


Checkpoint the agent so that the relevant model can be saved and deployed to Clipper. We save the name of the checkpoint file in `metadata.json` so the model container knows how to restore the policy checkpoint.

In [7]:
import os
import json
checkpoint_path = trainer.save()
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint_file = os.path.basename(checkpoint_path)
with open(os.path.join(checkpoint_dir, "metadata.json"), "w") as f:
    json.dump({"checkpoint": checkpoint_file}, f)

In [13]:
import tarfile
import io

model = io.BytesIO()
with tarfile.open(fileobj=model, mode="w:gz") as tar:
        tar.add(checkpoint_dir, arcname=os.path.basename(checkpoint_dir))
        

In [ ]:
encrypt_response = wave.EncryptMessage(
    wv.EncryptMessageParams(
        namespace=namespace,
        resource="models/pong",
        content=bytes(model.getvalue(),"utf8")))

In [ ]:
ciphertext = encrypt_response.ciphertext

decrypt_response = wave.DecryptMessage(wv.DecryptMessageParams(
        perspective= perspective,
        ciphertext= ciphertext,
        resyncFirst= True))
if resp.error.code != 0:
    raise Exception(resp.error.Message)

In [15]:
decryptedpath = "outputmodel"
decrypted_file = os.path.join(decryptedpath,checkpoint_file)
decryptedmodel = io.BytesIO(plaintext)
with tarfile.open(fileobj=decryptedmodel, mode="r:gz") as tar:
    tar.extractall(path=decryptedpath)

## Play Against the Policy

In this section, we will play Pong against the policy that we just trained. The game will be played in your browser, and the policy that we trained will be served by Clipper.

**EXERCISE:** Deploy your policy using Clipper. Follow the instructions that get printed below to play Pong against the deployed policy. You'll need to deploy all of the data that is saved in the directory `os.path.dirname(checkpoint_path)`.

Start by importing the `clipper_admin` library and use that to create a new Clipper instance to serve the policy.

When you create your ClipperConnection, you need to tell it how to communicate with the Docker service and Clipper. You can use the following command to get the Docker IP address. Use that address when you create your `ClipperConnection` in the next step.

In [ ]:
# Make logging work correctly in the Jupyter notebook
import logging
import sys
import subprocess32 as subprocess
logger = logging.getLogger()
logger.setLevel(logging.INFO)

from clipper_admin import DockerContainerManager, ClipperConnection
docker_ip = subprocess.check_output("./get_docker_ip.sh").strip()
clipper_conn = ClipperConnection(DockerContainerManager(docker_ip_address=docker_ip))
# Add a call to stop all in case you still have Clipper running from the earlier exercises
clipper_conn.stop_all()
clipper_conn.start_clipper()

Next, deploy the saved policy checkpoint to Clipper using a Docker image we created for this exercise (similar to the TensorFlow model container in the Clipper tutorial). If you're curious, you can find the custom model container code on [GitHub](https://github.com/ucbrise/risecamp/blob/077aa51078e2043d4d3d2d539e256c30c259678e/rl_and_pong/pong_model_container.py).

In [ ]:
import os
model_name = "pong-policy"
app_name = "pong"
clipper_conn.build_and_deploy_model(
    name=model_name,
    version=1,
    input_type="doubles",
    model_data_path=os.path.dirname(checkpoint_path),
    base_image="clipper/risecamp-pong-container"
)

Finally, register a Clipper application and link it the deployed policy model.

In [ ]:
app_name = "pong"
clipper_conn.register_application(name=app_name, default_output="0", input_type="doubles", slo_micros=100000)
clipper_conn.link_model_to_app(app_name=app_name, model_name=model_name)

Now that you have deployed your policy to Clipper, you will start a Pong application that will let you play against your policy in the browser.

When you start the application, you need to tell it where Clipper is running in order for the Pong application to request predictions from Clipper. `ClipperConnection` provides the `get_query_addr()` method to get the IP address and port on which Clipper is listening for incoming prediction requests.

In [ ]:
clipper_addr = clipper_conn.get_query_addr()
print("Clipper address: {}".format(clipper_addr))

Now you can start the Pong webserver. It will print out the URL it's running on after it starts. Copy and paste that URL into your browser and press "1" to play against your policy!

In [ ]:
import subprocess32 as subprocess
server_handle = subprocess.Popen(["./start_webserver.sh", clipper_addr], stdout=subprocess.PIPE)
print(server_handle.stdout.readline().strip())

## Deploy a New Policy

The first policy that you deploy probably won't be a very strong competitor, especially if you only trained it for a few iterations. Try training it for more iterations and deploying the new policy to Clipper. Clipper will automatically switch the Pong application to query the new version of the policy. You don't need to reload the page or even restart the game.

For your convenience, we've copied the relevant cells from above to train the policy for more iterations and deploy it Clipper. You can run this cell as many times as you want. Don't forget to increment the version number of the model each time you deploy to Clipper.

In [ ]:
# Train for more iterations
for i in range(50):
    result = agent.train()
    
# Save the new policy
checkpoint_path = agent.save()
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint_file = os.path.basename(checkpoint_path)
with open(os.path.join(checkpoint_dir, "metadata.json"), "w") as f:
    json.dump({"checkpoint": checkpoint_file}, f)
    
# Deploy the new policy to Clipper.
clipper_conn.build_and_deploy_model(
    name=model_name,
    version=2, # If you run this more than once, don't forget to keep updating the version.
    input_type="doubles",
    model_data_path=os.path.dirname(checkpoint_path),
    base_image="clipper/risecamp-pong-container"
)